
# HDA Flowsheet Costing


## Note

This tutorial will demonstrate adding rigorous process costing to the two HDA examples, the basic [HDA with Flash](../../Tutorials/Basics/HDA_flowsheet_solution_testing.ipynb) and a comparison with the [HDA with Distillation](HDA_flowsheet_with_distillation_solution_testing.ipynb).


## Learning outcomes


- Import external pre-built steady-state flowsheets using the IDAES unit model library
- Define and add costing blocks using the IDAES Generic Costing Framework
- Fomulate and solve a process economics optimization problem
    - Defining an objective function
    - Setting variable bounds
    - Adding additional constraints 


## Problem Statement

Hydrodealkylation is a chemical reaction that often involves reacting
an aromatic hydrocarbon in the presence of hydrogen gas to form a
simpler aromatic hydrocarbon devoid of functional groups. In this
example, toluene will be reacted with hydrogen gas at high temperatures
 to form benzene via the following reaction:

**C<sub>6</sub>H<sub>5</sub>CH<sub>3</sub> + H<sub>2</sub> → C<sub>6</sub>H<sub>6</sub> + CH<sub>4</sub>**


This reaction is often accompanied by an equilibrium side reaction
which forms diphenyl, which we will neglect for this example.

This example is based on the 1967 AIChE Student Contest problem as
present by Douglas, J.M., Chemical  Design of Chemical Processes, 1988,
McGraw-Hill.

Users may refer to the prior examples linked at the top of this notebook for detailed process descriptions of the two HDA configurations. As before, the properties required for this module are defined in

- `hda_ideal_VLE.py`
- `idaes.models.properties.activity_coeff_models.BTX_activity_coeff_VLE`
- `hda_reaction_kinetic.py`

Additionally, we will be importing externally-defined flowsheets for the two HDA configurations from

- `hda_flowsheets.py`

## Import and run HDA Flowsheets
First, we will generate solved flowsheets for each HDA model. The external scripts build and set inputs for the flowsheets, initialize unit models and streams, and solve the flowsheets before returning the model objects. Note that the HDA flowsheets contain all unit models and stream connections, and no costing equations:

In [ ]:
# source file for external flowsheets
from hda_flowsheets import hda_with_flash, hda_with_distillation

In [ ]:
# alias 'm' for hda model with second flash unit
m = hda_with_flash(tee=False)

In [ ]:
# alias 'n' for hda model with distillation column
n = hda_with_distillation(tee=False)

## IDAES Generic Costing Framework
IDAES provides a rigorous costing package for process economics calculations, based on methods from the following source:

*Process and Product Design Principles: Synthesis, Analysis, and Evaluation*. Seider, Seader, Lewin, Windagdo, 3rd Ed. John Wiley and Sons Chapter 22. Cost Accounting and Capital Cost Estimation 22.2 Cost Indexes and Capital Investment.

Currently, IDAES supports calculation of capital costing for a wide array of unit operations:

- Pressure Changers (Compressor, Turbine, Pump, generic PressureChanger)
- Temperature Changers (Heater, HeatExchanger, HeatExchangerNTU)
- Vertical and Horizontal Vessels (Flash, CSTR, PFR, StoichiometricReactor)

The framework includes factors for vessel shell thickness and dimensions, material properties for equipment shells and tubes, and tray types for distillation columns. The framework does not yet support heat exchangers requiring both shell and tube area (HX1D).

For capital costs calculations, IDAES supports the following options:
- Heat exchanger type:
    - floating head, fixed head, "U tube", Kettle evaporator
- Heat exchanger material:
    - pure Carbon Steel, Stainless Steel, Cr Mo Steel, Monel or Titanium; Carbon Steel alloy with Brass, Stainless Steel, Monel, Titanium or Cr Mo Steel; 
- Heat exchanger tube lengths:
    - 8, 12, 16 or 20 feet
- Vessel materials
    - Carbon Steel, Low Alloy Steel, Stainless Steel 304, Stainless Stell 316, Carpenter 20CB3, Nickel 200, Money 400, Inconel 600, Incoloy 825, Titanium
- Column tray type:
    - Sieve, Valve, Bubble Cap
- Column tray material:
    - Carbon Steel, Stainless Stell 303, Stainless Stell 316, Carpenter 20CB3, Monel
- Heater material:
    - Carbon Steel, Cr Mo Steel, Stainless Steel
- Heater source:
    - Fuel, Reformer, Pyrolysis, Hot Water, Salts, Dowtherm A, Steam Boiler
- Compressor type:
    - Centrifugal, Reciprocating, Screw
- Compressor drive type:
    - Electric Motor, Steam Turbine, Gas Turbine
- Compressor material:
    - Carbon Steel, Stainless Steel, Nickel Alloy
- Pump Material:
    - Cast Iron, Ductile Iron, Cast Steel, Bronze, Stainless Steel, Hastelloy C, Monel, Nickel, Titanium, Ni Al Bronze, Carbon Steel
- Pump type:
    - Centrigual, External Gear, Reciprocating
- Pump motor type:
    - Open, Enclosed, Explosion Proof
- Fan type:
    - Centrifugal Backward, Centrifugal Straight, Vane Axial, Tube Axial
- Fan material:
    - Carbon Steel, Fiberglass, Stainless Steel, Nickel Alloy
- Blower type:
    - Centrifugal, Rotary
- Blower material:
    - Carbon Steel, Aluminum, Fiberglass, Stainless Steel, Nickel Alloy


The framework contains the following flowsheet costing methods and default options under the "SSLWCosting" class:
- build_global_params() sets base currency and period, and other optional parameters
- build_process_costs() grants access to aggregate capital, fixed, variable and flow costs for process wide costing
- initialize_build() allows for optional initialization of costing blocks when required; aggregate costs are already initialized elsewhere
- cost_heat_exchanger(blk,hx_type=HXType.Utube, material_type=HXMaterial.StainlessSteelStainlessSteel, tube_length=HXTubeLength.TwelveFoot, integer=True)
- def cost_vessel( blk, vertical=False, material_type=VesselMaterial.CarbonSteel, shell_thickness=1.25 * pyo.units.inch, weight_limit=1, aspect_ratio_range=1, include_platforms_ladders=True, vessel_diameter=None, vessel_length=None, number_of_units=1, number_of_trays=None, tray_material=TrayMaterial.CarbonSteel, tray_type=TrayType.Sieve)
- def cost_vertical_vessel(blk, material_type=VesselMaterial.CarbonSteel, shell_thickness=1.25 * pyo.units.inch, weight_limit=1, aspect_ratio_range=1, include_platforms_ladders=True, vessel_diameter=None, vessel_length=None, number_of_units=1, number_of_trays=None, tray_material=TrayMaterial.CarbonSteel, tray_type=TrayType.Sieve)
- def cost_horizontal_vessel(blk, material_type=VesselMaterial.CarbonSteel, shell_thickness=1.25 * pyo.units.inch, include_platforms_ladders=True, vessel_diameter=None, vessel_length=None, number_of_units=1)
- def cost_fired_heater(blk, heat_source=HeaterSource.Fuel, material_type=HeaterMaterial.CarbonSteel, integer=True)
- def cost_compressor(blk, compressor_type=CompressorType.Centrifugal, drive_type=CompressorDriveType.ElectricMotor, material_type=CompressorMaterial.StainlessSteel, integer=True)
- def cost_fan(blk, fan_type=FanType.CentrifugalBackward, fan_head_factor=1.45, material_type=FanMaterial.StainlessSteel, integer=True)
- def cost_blower(blk, blower_type=BlowerType.Centrifugal, material_type=BlowerMaterial.StainlessSteel, integer=True)
- def cost_turbine(blk, integer=True)
- def cost_pump(blk, pump_type=PumpType.Centrifugal, material_type=PumpMaterial.StainlessSteel, pump_type_factor=1.4, motor_type=PumpMotorType.Open, integer=True)
- def cost_pressure_changer(blk, mover_type="compressor", **kwargs)

As a convenience for users, common unit models are mapped with appropriate costing methods. Therefore, for quick costing with defaults, users may import the `unit_mapping` dictionary and pass the unit model type as below:

*unit_mapping = {  
        CSTR: cost_vertical_vessel,  
        Compressor: cost_compressor,  
        Flash: cost_vertical_vessel,  
        Heater: cost_fired_heater,  
        HeatExchanger: cost_heat_exchanger,  
        HeatExchangerNTU: cost_heat_exchanger,  
        PFR: cost_horizontal_vessel,  
        PressureChanger: cost_pressure_changer,  
        Pump: cost_pump,  
        StoichiometricReactor: cost_horizontal_vessel,  
        Turbine: cost_turbine,  
    }*

*costing_blk = unit_mapping[type(blk)]*

The mapping is inheritance aware, e.g. pumps will try costing with cost_pump() and fall back to cost_pressure_changer.

## Add Operating Cost Equations
Before adding capital costing blocks, we will add operating cost equations taken from the prior examples. The examples assume constant cooling and heating coefficients over an annual cost basis. The IDAES Generic Costing Framework does not currently support variable cost calculations.

In [ ]:
# required imports
from pyomo.environ import Expression

# operating costs for HDA with second flash (model m)
m.fs.cooling_cost = Expression(expr=0.212e-7 * (-m.fs.F101.heat_duty[0]) +
                                   0.212e-7 * (-m.fs.R101.heat_duty[0]))
m.fs.heating_cost = Expression(expr=2.2e-7 * m.fs.H101.heat_duty[0] +
                                   1.9e-7 * m.fs.F102.heat_duty[0])
m.fs.operating_cost = Expression(expr=(3600 * 24 * 365 *
                                           (m.fs.heating_cost +
                                            m.fs.cooling_cost)))

# operating costs for HDA with distillation (model n)
n.fs.cooling_cost = Expression(expr=0.25e-7 * (-n.fs.F101.heat_duty[0]) +
                                0.2e-7 * (-n.fs.D101.condenser.heat_duty[0]))

n.fs.heating_cost = Expression(expr=2.2e-7 * n.fs.H101.heat_duty[0] +
                                1.2e-7 * n.fs.H102.heat_duty[0] +
                                1.9e-7 * n.fs.D101.reboiler.heat_duty[0])

n.fs.operating_cost = Expression(expr=(3600 * 24 * 365 *
                                (n.fs.heating_cost + n.fs.cooling_cost)))   

## Add Capital Costing
Below, we will add add rigorous capital costing blocks to the imported flowsheets and evaluate the economic impact of replacing the second Flash with a Distillation column. First, let's import the relevant costing methods:

In [ ]:
# Import costing methods - classes, heaters, vessels, compressors, columns
from idaes.models.costing.SSLW import (
    SSLWCosting,
    SSLWCostingData,
    VesselMaterial,
    TrayType,
    TrayMaterial,
    HeaterMaterial,
    HeaterSource,
    CompressorDriveType,
    CompressorMaterial,
    CompressorType,
)
from idaes.core import UnitModelCostingBlock

Next, we will build the main costing block for each flowsheet, calling the main costing class to define the blocks:

In [ ]:
# costing block for HDA with second flash (model m)
m.fs.costing = SSLWCosting()

# costing block for HDA with distillation (model n)
n.fs.costing = SSLWCosting()

Finally, we will build the relevant costing blocks for the equipment we wish to cost. Note how the costing block, methods and flags are passed as arguments in the costing block call itself. Each unit model will have a single costing block, but each flowsheet model (m and n) will also have a single costing block for flowsheet-level properties.

Users should note that IDAES costing methods support a wide array of heating sources (e.g. fired, steam boiler, hot water) and do not support direct capital costing of coolers. If users wish to cost Heater units acting as coolers, it is necessary to cost a "dummy" [0D shell and tube exchanger](https://idaes-pse.readthedocs.io/en/stable/reference_guides/model_libraries/generic/unit_models/heat_exchanger.html) with appropriate aliased hot stream properties and proper cooling water properties. This is not demonstrated here, as the HDA examples take advantage of Flash and Condenser operations to recover liquid product.

Capital costing is independent of unit model connections, and building cost equations may be done piecewise in this fashion. Default options are passed explicitly to demonstrate proper syntax and usage. Now that all required properties are defined, let's cost our models connecting costing blocks, methods and unit models in each flowsheet.

### Flexibility of Costing Block Definitions
IDAES supports many ways to define batches of costing blocks, and several are shown in the example. Users may employ whichever method fits their modeling needs for explicit or concise code. In the code below, note how the unit model itself is never passed to the costing method; when the full model is executed, the costing block will automatically connect its parent block with child equation blocks.

To demonstrate proper usage, the Compressor costing blocks are defined below. For this example, the compressors assume isothermal thermodynamics and are too simple for the costing module. As shown below, attempting to cost the compressors throws a `ValueError` warning:

In [ ]:
# costing for compressors - m.fs.C101, n.fs.C101

try:
    m.fs.C101.costing = UnitModelCostingBlock(
        default={
            "flowsheet_costing_block": m.fs.costing,
            "costing_method": SSLWCostingData.cost_compressor,
            "costing_method_arguments": {
                "compressor_type": CompressorType.Centrifugal,
                "drive_type": CompressorDriveType.ElectricMotor,
                "material_type": CompressorMaterial.StainlessSteel,
            },
        }
    )
except:
    delattr(m.fs.C101, "costing")

In [ ]:
try:
    n.fs.C101.costing = UnitModelCostingBlock(
        default={
            "flowsheet_costing_block": n.fs.costing,
            "costing_method": SSLWCostingData.cost_compressor,
            "costing_method_arguments": {
                "compressor_type": CompressorType.Centrifugal,
                "drive_type": CompressorDriveType.ElectricMotor,
                "material_type": CompressorMaterial.StainlessSteel,
            },
        }
    )
except:
    delattr(n.fs.C101, "costing")

To demonstrate the flexibility of the framework, let's define all Heater costing blocks sequentially using a loop:

In [ ]:
# costing for heaters - m.fs.H101, n.fs.H101, n.fs.H102

# loop over units
for unit in [m.fs.H101, n.fs.H101, n.fs.H102]:
    unit.costing = UnitModelCostingBlock(
        default={
            "flowsheet_costing_block": unit.parent_block().costing,
            "costing_method": SSLWCostingData.cost_fired_heater,
            "costing_method_arguments": {
                "material_type": HeaterMaterial.CarbonSteel,
                "heat_source": HeaterSource.Fuel,
            }
        }
    )

The costing module provides a `unit_mapping` dictionary linking generic unit model classes with recommended costing methods. In this example, CSTR, StoichiometricReactor and Flash vessels utilize different vessel costing methods with similar arguments. The diameter and length attributes need to exist, and we add them if they don't exist already. The `unit_mapping` method provides an opportunity to automatically select the correct vessel orientation (vertical or horizontal) based on the unit type:

In [ ]:
import math
from pyomo.environ import Var, Constraint, units as pyunits, Param, value
from idaes.models.unit_models import CSTR, StoichiometricReactor, Flash
# map unit models to unit classes
# will pass to unit_mapping which calls costing methods based on unit class
unit_class_mapping = {m.fs.R101: StoichiometricReactor,
                      m.fs.F101: Flash,
                      m.fs.F102: Flash,
                      n.fs.R101: CSTR,
                      n.fs.F101: Flash}

# costing for vessels - m.fs.R101, m.fs.F101, m.fs.F102, n.fs.R101, n.fs.F101

# loop over units
for unit in [m.fs.R101, m.fs.F101, m.fs.F102, n.fs.R101, n.fs.F101]:
    # get correct unit class for unit model
    unit_class = unit_class_mapping[unit]
    
    # add dimension variables and constraint if they don't exist
    if not hasattr(unit, "diameter"):
        unit.diameter = Var(initialize=1, units=pyunits.m)
    if not hasattr(unit, "length"):
        unit.length = Var(initialize=1, units=pyunits.m)
    if hasattr(unit, "volume"):  # if volume exists, set diameter from volume
        unit.volume_eq = Constraint(expr=unit.volume[0] == unit.length * unit.diameter**2 * 0.25 * math.pi)
    else:  # fix diameter directly
        unit.diameter.fix(0.2214 * pyunits.m)
    # either way, fix L/D to calculate L from D
    unit.L_over_D = Constraint(expr=unit.length == 3 * unit.diameter)
        
    # define vessel costing
    unit.costing = UnitModelCostingBlock(
        default={
            "flowsheet_costing_block": unit.parent_block().costing,
            "costing_method": SSLWCostingData.unit_mapping[unit_class],
            "costing_method_arguments": {
                "material_type": VesselMaterial.CarbonSteel,
                "shell_thickness": 1.25 * pyunits.inch
                
            }
        }
    )

Finally, we will define costing for the distillation column:

In [ ]:
# costing for column - n.fs.D101

# define column dimensions - same volume as m.fs.F102 with L/D of 5
n.fs.D101.diameter = Param(initialize=0.1715, units=pyunits.m)
n.fs.D101.length = Param(initialize=0.8575, units=pyunits.m)

n.fs.D101.costing = UnitModelCostingBlock(
        default={
            "flowsheet_costing_block": n.fs.costing,
            "costing_method": SSLWCostingData.cost_vertical_vessel,
            "costing_method_arguments": {
                "material_type": VesselMaterial.CarbonSteel,
                "shell_thickness": 1.25 * pyunits.inch,
                "include_platforms_ladders": True,
                "number_of_trays": n.fs.D101.config.number_of_trays,
                "tray_material": TrayMaterial.CarbonSteel,
                "tray_type": TrayType.Sieve
                
            }
        }
    )

## Solve Flowsheet Costing Blocks
Now, we may solve the full flowsheet for all costing blocks:

In [ ]:
# define solver
from idaes.core.solvers import get_solver
solver = get_solver()
solver.options = {"halt_on_ampl_error": 'yes'}

# Check that the degrees of freedom is zero
from idaes.core.util.model_statistics import degrees_of_freedom
assert degrees_of_freedom(m) == 0
assert degrees_of_freedom(n) == 0

In [ ]:
results_m = solver.solve(m, tee=True, symbolic_solver_labels=True)

In [ ]:
# Check solver solve status and physical units consistency for model m
from pyomo.environ import TerminationCondition
from pyomo.util.check_units import assert_units_consistent
assert results_m.solver.termination_condition == TerminationCondition.optimal
assert_units_consistent(m)

In [ ]:
results_n = solver.solve(n, tee=True, symbolic_solver_labels=True)

In [ ]:
# Check solver solve status and physical units consistency for model n
assert results_n.solver.termination_condition == TerminationCondition.optimal
assert_units_consistent(n)

## Results Comparison and Visualization
For the two flowsheets above, let's sum the total operating and capital costs of each scenario. We will display overall process economics results and compare the two flowsheets:

In [ ]:
# imports and data gathering
from matplotlib import pyplot as plt
plt.style.use('dark_background')  # if using browser in dark mode, uncomment this line
import numpy as np
import pandas as pd

# automatically get units that we costed - this will exclude C101 for both flowsheets

two_flash_unitlist = [getattr(m.fs, unit) for unit in dir(m.fs)
                      if hasattr(getattr(m.fs, unit), "costing")]
distillation_unitlist = [getattr(n.fs, unit) for unit in dir(n.fs)
                       if hasattr(getattr(n.fs, unit), "costing")]

In [ ]:
# compare equipment purchase costs (actual capital costs)

two_flash_capcost = {unit.name: value(unit.costing.capital_cost/1e3)
                     for unit in two_flash_unitlist}
distillation_capcost = {unit.name: value(unit.costing.capital_cost/1e3)
                      for unit in distillation_unitlist}

two_flash_capdf = pd.DataFrame(list(two_flash_capcost.items()),
                            columns=['Equipment','Two Flash']).set_index('Equipment')
distillation_capdf = pd.DataFrame(list(distillation_capcost.items()),
                             columns=['Equipment','Distillation']).set_index('Equipment')

# add dataframes, merge same indices, replace NaNs with 0s, and transpose
capcosts = two_flash_capdf.add(distillation_capdf, fill_value=0).fillna(0).transpose()

# sort according to an easier order to view
capcosts = capcosts[['fs.H101', 'fs.R101', 'fs.F101', 'fs.F102', 'fs.D101', 'fs.H102']]

print('Costs in $1000:')
display(capcosts)  # view dataframe before plotting

capplot = capcosts.plot(kind='bar', stacked=True, title='HDA Capital Costs', ylabel='$1000')

In [ ]:
# compare operating costs (per year)

two_flash_opcost = {'cooling': value(3600 * 24 * 365 * m.fs.cooling_cost/1e3),
                    'heating': value(3600 * 24 * 365 * m.fs.heating_cost/1e3)}
distillation_opcost = {'cooling': value(3600 * 24 * 365 * n.fs.cooling_cost/1e3),
                     'heating': value(3600 * 24 * 365 * n.fs.heating_cost/1e3)}

two_flash_opdf = pd.DataFrame(list(two_flash_opcost.items()),
                            columns=['Utilities','Two Flash']).set_index('Utilities')
distillation_opdf = pd.DataFrame(list(distillation_opcost.items()),
                             columns=['Utilities','Distillation']).set_index('Utilities')

# add dataframes, merge same indices, replace NaNs with 0s, and transpose
opcosts = two_flash_opdf.add(distillation_opdf, fill_value=0).fillna(0).transpose()

print('Costs in $1000:')
display(opcosts)  # view dataframe before plotting

opplot = opcosts.plot(kind='bar', stacked=True, title='HDA Operating Costs', ylabel='$1000/year')

In [ ]:
# compare total costs (capital costs and operating costs)

two_flash_totcost = {'capital': sum(two_flash_capcost[idx] for idx in two_flash_capcost),
                    'operating': value(m.fs.operating_cost)/1e3}
distillation_totcost = {'capital': sum(distillation_capcost[idx] for idx in distillation_capcost),
                     'operating': value(n.fs.operating_cost)/1e3}

two_flash_totdf = pd.DataFrame(list(two_flash_totcost.items()),
                            columns=['Costs','Two Flash']).set_index('Costs')
distillation_totdf = pd.DataFrame(list(distillation_totcost.items()),
                             columns=['Costs','Distillation']).set_index('Costs')

# add dataframes, merge same indices, replace NaNs with 0s, and transpose
totcosts = two_flash_totdf.add(distillation_totdf, fill_value=0).fillna(0).transpose()

print('Costs in $1000:')
display(totcosts)  # view dataframe before plotting

totplot = totcosts.plot(kind='bar', stacked=True, title='HDA Total Plant Cost (TPC)', ylabel='$1000/year')

Finally, let's compare the total cost on a production basis. This will account for the greater efficiency provided by the distillation column relative to the less-expensive second flash unit:

In [ ]:
two_flash_cost = value(1e3*sum(two_flash_totcost[idx] for idx in two_flash_totcost))
two_flash_prod = value(m.fs.F102.vap_outlet.flow_mol_phase_comp[0, "Vap", "benzene"] * 365 * 24 * 3600)
distillation_cost = value(1e3*sum(distillation_totcost[idx] for idx in distillation_totcost))
distillation_prod = value(n.fs.D101.condenser.distillate.flow_mol[0] * 365 * 24 * 3600)

print(f"Two flash case over one year: ${two_flash_cost/1e3:0.0f}K / {two_flash_prod/1e3:0.0f} kmol benzene = ${two_flash_cost/(two_flash_prod/1e3):0.2f} per kmol benzene produced")
print(f"Distillation case over one year: ${distillation_cost/1e3:0.0f}K / {distillation_prod/1e3:0.0f} kmol benzene = ${distillation_cost/(distillation_prod/1e3):0.2f} per kmol benzene produced")

# Summary
In this example, IDAES Generic Costing Framework methods were applied to two HDA flowsheets for capital cost estimation. The costing blocks calls showcased multiple methods to define unit costing, demonstrating the flexibility and best practice of the costing framework. In the basic examples, the two-flash HDA did not include costing and the distillation HDA estimated a reactor capital cost comprising 3.3% of the total plant cost (TPC). With more rigorous costing, IDAES obtained total capital costs of 8.5% TPC (two flash HDA) and 9.6% (distillation HDA) and better modeled the impact of equipment cost on process economics. As printed above, the IDAES Generic Costing Framework confirmed that replacing the second flash drum with a distillation column results in increased equipment costs, increased production and decreased cost per unit product.